<a href="https://colab.research.google.com/github/ricardofoltran/assistente-voz-whisper-chatgpt/blob/main/assistente_voz_whisper_chatgpt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##  Passo 1 - Definição de Idioma


In [2]:
language = 'pt'  # pt = português | en = inglês | es = espanhol

## Passo 2 – Gravação de Áudio no Google Colab 🎤

🎧 Observação sobre a gravação de áudio

Durante o desenvolvimento deste projeto, um áudio de teste pode já estar salvo no notebook.

Para utilizar **sua própria voz**, execute novamente a célula de gravação de áudio abaixo.  
Ao executar a célula, o navegador solicitará permissão de acesso ao microfone.

➡️ Sempre que quiser gravar um novo áudio, basta **executar novamente a célula de gravação**.


In [ ]:
# Referência: https://gist.github.com/korakot/c21c3476c024ad6d56d5f48b0bca92be

from IPython.display import Audio, display, Javascript
from google.colab import output
from base64 import b64decode

# Código JavaScript para gravar áudio do usuário usando a "MediaStream Recording API"
RECORD = """
const sleep  = time => new Promise(resolve => setTimeout(resolve, time))
const b2text = blob => new Promise(resolve => {
  const reader = new FileReader()
  reader.onloadend = e => resolve(e.srcElement.result)
  reader.readAsDataURL(blob)
})
var record = time => new Promise(async resolve => {
  stream = await navigator.mediaDevices.getUserMedia({ audio: true })
  recorder = new MediaRecorder(stream)
  chunks = []
  recorder.ondataavailable = e => chunks.push(e.data)
  recorder.start()
  await sleep(time)
  recorder.onstop = async ()=>{
    blob = new Blob(chunks)
    text = await b2text(blob)
    resolve(text)
  }
  recorder.stop()
})
"""

def record_audio(seconds=5):
    display(Javascript(RECORD))
    print("🎧 Gravando áudio...")

    audio_data = output.eval_js(f'record({seconds * 1000})')
    audio_bytes = b64decode(audio_data.split(',')[1])
    with open('/content/audio_usuario.wav', 'wb') as f:
        f.write(audio_bytes)

    return '/content/audio_usuario.wav'

# Call the function and store the result before displaying
audio_file_path = record_audio(5) # Records for 5 seconds
display(Audio(audio_file_path))

## Passo 3 – Reconhecimento de Fala com Whisper 🧠



In [10]:
!pip install git+https://github.com/openai/whisper.git -q

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [9]:
import whisper

model = whisper.load_model("small")

100%|████████████████████████████████████████| 461M/461M [00:01<00:00, 252MiB/s]


In [12]:
result = model.transcribe(audio_file_path, fp16=False, language=language)
transcription = result["text"]

print("📝 Transcrição:")
print(transcription)

📝 Transcrição:
 Um, dois, três, testando, opa, opa!


## 🔐 Passo 4 – Integração com ChatGPT 💬

Nesta etapa, o projeto se conecta à **API da OpenAI (ChatGPT)** para gerar uma resposta inteligente a partir do texto transcrito pelo Whisper.

⚠️ **Importante – Segurança da API Key**

Por motivos de segurança, **nenhuma API Key é fornecida neste projeto**.

Para executar corretamente este passo, **você deve utilizar a sua própria API Key da OpenAI**.

### Como configurar:

1. Acesse o site da OpenAI e gere sua API Key:
   - https://platform.openai.com/account/api-keys

2. No código abaixo, substitua o valor:
   ```python
   'SUA_API_KEY_AQUI'


In [13]:
!pip install openai -q

In [14]:
import os
import openai

os.environ['OPENAI_API_KEY'] = 'SUA_API_KEY_AQUI'
openai.api_key = os.environ['OPENAI_API_KEY']

In [ ]:
from openai import OpenAI

# Initialize the OpenAI client (API key is picked up from environment variables)
client = OpenAI()

response = client.chat.completions.create(
    model="gpt-4",
    messages=[
        {"role": "system", "content": "Você é um assistente de voz inteligente."},
        {"role": "user", "content": transcription}
    ]
)

chatgpt_response = response.choices[0].message.content

print("🤖 Resposta do ChatGPT:")
print(chatgpt_response)

##  Passo 5 – Converter Resposta em Áudio (Text-to-Speech) 🔊

In [20]:
!pip install gTTS -q

In [ ]:
from gtts import gTTS

tts = gTTS(
    text=chatgpt_response,
    lang=language,
    slow=False
)

response_audio_path = "/content/resposta_assistente.wav"
tts.save(response_audio_path)

In [ ]:
display(Audio(response_audio_path, autoplay=True))